In [2]:
# Célula 1: Importações e setup
import pandas as pd
import numpy as np
from tk_data_preparer.core import (
    CleanConfig, FilterConfig, DedupConfig, PipelineConfig, 
    DataPreparationPipeline, clean_columns, filter_by_length, remove_duplicates
)

print("Módulos importados com sucesso!")

Módulos importados com sucesso!


In [3]:
# Célula 2: Criar dados mock realistas
np.random.seed(42)
data = {
    'nome_completo': ['  João Silva  ', 'MARIA SANTOS', 'joão silva', 'Ana Costa', '  MARIA SANTOS  ', None, 'Pedro Alves', 'Lu'],
    'email': ['JOÃO@EMAIL.COM', 'maria@email.com', 'joão@email.com', 'ana@email.com', 'MARIA@EMAIL.COM', 'test@email.com', 'pedro@email.com', 'lu@email.com'],
    'telefone': ['(11) 99999-9999', '(21) 88888-8888', '(11) 99999-9999', '(31) 77777-7777', '(21) 88888-8888', None, '(41) 66666-6666', '(51) 55555-5555'],
    'idade': [25, 30, 25, 35, 30, 28, 150, 17],  # idade inválida (150)
    'salario': [5000.0, 6000.5, 5000.0, 7500.0, 6000.5, 3000.0, 10000.0, 2000.0],
    'departamento': ['TI', 'RH', 'TI', 'Vendas', 'RH', 'TI', 'TI', 'Estágio'],
    'data_admissao': ['2020-01-15', '2019-03-20', '2020-01-15', '2018-06-10', '2019-03-20', '2021-02-01', '2017-12-05', '2023-01-10']
}

df_original = pd.DataFrame(data)
print("Dados originais criados:")
print(f"Shape: {df_original.shape}")
display(df_original)

Dados originais criados:
Shape: (8, 7)


,nome_completo,email,telefone,idade,salario,departamento,data_admissao
0,João Silva,JOÃO@EMAIL.COM,(11) 99999-9999,25,5000.0,TI,2020-01-15
1,MARIA SANTOS,maria@email.com,(21) 88888-8888,30,6000.5,RH,2019-03-20
2,joão silva,joão@email.com,(11) 99999-9999,25,5000.0,TI,2020-01-15
3,Ana Costa,ana@email.com,(31) 77777-7777,35,7500.0,Vendas,2018-06-10
4,MARIA SANTOS,MARIA@EMAIL.COM,(21) 88888-8888,30,6000.5,RH,2019-03-20
5,None,test@email.com,None,28,3000.0,TI,2021-02-01
6,Pedro Alves,pedro@email.com,(41) 66666-6666,150,10000.0,TI,2017-12-05
7,Lu,lu@email.com,(51) 55555-5555,17,2000.0,Estágio,2023-01-10


In [4]:
# Célula 3: Análise inicial dos dados
print("Análise inicial:")
print(f"Total de registros: {len(df_original)}")
print(f"Valores nulos por coluna:")
print(df_original.isnull().sum())
print(f"\nTipos de dados:")
print(df_original.dtypes)

Análise inicial:
Total de registros: 8
Valores nulos por coluna:
nome_completo    1
email            0
telefone         1
idade            0
salario          0
departamento     0
data_admissao    0
dtype: int64

Tipos de dados:
nome_completo     object
email             object
telefone          object
idade              int64
salario          float64
departamento      object
data_admissao     object
dtype: object


In [6]:
# Célula 4: Testar módulo de cleaning individualmente
print("Testando módulo de limpeza...")
clean_config = CleanConfig(
    strip=True,
    collapse_whitespace=True,
    unicode_normalization='NFKC',
    case='title',  # João Silva, Maria Santos
    empty_to_na=True
)

df_cleaned, clean_metrics = clean_columns(df_original, columns=['nome_completo', 'email', 'departamento'])

print("Limpeza concluída!")
print(f"Métricas: {clean_metrics}")
print("\nAntes vs Depois:")
print(pd.DataFrame({
    'Antes': df_original['nome_completo'].head(3),
    'Depois': df_cleaned['nome_completo'].head(3)
}))

Testando módulo de limpeza...
Limpeza concluída!
Métricas: {'total_whitespace_removed': 8, 'per_column_whitespace_removed': {'nome_completo': 8, 'email': 0, 'departamento': 0}, 'cells_modified': 2, 'empty_strings_to_na': 1, 'columns_processed': ['nome_completo', 'email', 'departamento']}

Antes vs Depois:
            Antes        Depois
0    João Silva      João Silva
1    MARIA SANTOS  MARIA SANTOS
2      joão silva    joão silva


In [7]:
# Célula 5: Testar módulo de filtering individualmente
print("\nTestando módulo de filtragem...")
filter_config = FilterConfig(min_length=5, drop_na=True)
df_filtered, filter_metrics = filter_by_length(df_cleaned, 'nome_completo', filter_config)

print("Filtragem concluída!")
print(f"Métricas: {filter_metrics}")
print(f"Registros após filtro: {len(df_filtered)}")


Testando módulo de filtragem...
Filtragem concluída!
Métricas: {'removed_rows': 2, 'remaining_rows': 6, 'min_length': 5, 'drop_na': True}
Registros após filtro: 6


In [9]:
# Célula 6: Testar módulo de deduplication individualmente
print("\nTestando módulo de deduplicação...")
dedup_config = DedupConfig(keep='first', case_sensitive=False, normalize_unicode=True)
df_dedup, dedup_metrics = remove_duplicates(df_filtered, 'email', dedup_config)

print("Deduplicação concluída!")
print(f"Métricas: {dedup_metrics}")
print(f"Registros únicos: {len(df_dedup)}")


Testando módulo de deduplicação...
Deduplicação concluída!
Métricas: {'removed_duplicates': 2, 'remaining_rows': 4, 'keep_strategy': 'first', 'case_sensitive': False, 'normalize_unicode': True}
Registros únicos: 4


In [10]:
# Célula 7: Pipeline COMPLETO
print("\nTestando PIPELINE COMPLETO...")
pipeline_config = PipelineConfig(
    cleaning=CleanConfig(
        strip=True,
        collapse_whitespace=True,
        unicode_normalization='NFKC',
        case='title',
        empty_to_na=True
    ),
    filtering=FilterConfig(min_length=5, drop_na=True),
    deduplication=DedupConfig(
        keep='first',
        case_sensitive=False,
        normalize_unicode=True
    )
)

pipeline = DataPreparationPipeline(pipeline_config)
resultado = pipeline.run(
    df_original, 
    target_column='nome_completo',
    columns_to_clean=['nome_completo', 'email', 'departamento']
)

print("PIPELINE COMPLETO CONCLUÍDO!")
print(f"Registros originais: {len(df_original)}")
print(f"Registros finais: {len(resultado.dataframe)}")
print(f"Redução: {len(df_original) - len(resultado.dataframe)} registros removidos")


Testando PIPELINE COMPLETO...
PIPELINE COMPLETO CONCLUÍDO!
Registros originais: 8
Registros finais: 4
Redução: 4 registros removidos


In [11]:
# Célula 8: Análise dos resultados
print("\nMétricas detalhadas do pipeline:")
for etapa, metrics in resultado.metrics.items():
    print(f"\n{etapa.upper()}:")
    for key, value in metrics.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for k, v in value.items():
                print(f"    {k}: {v}")
        else:
            print(f"  {key}: {value}")


Métricas detalhadas do pipeline:

CLEANING:
  total_whitespace_removed: 8
  per_column_whitespace_removed:
    nome_completo: 8
    email: 0
    departamento: 0
  cells_modified: 2
  empty_strings_to_na: 1
  columns_processed: ['nome_completo', 'email', 'departamento']

FILTERING:
  removed_rows: 2
  remaining_rows: 6
  min_length: 5
  drop_na: True

DEDUPLICATION:
  removed_duplicates: 2
  remaining_rows: 4
  keep_strategy: first
  case_sensitive: False
  normalize_unicode: True


In [12]:
# Célula 9: Visualização dos dados transformados
print("\nDados finais após pipeline completo:")
display(resultado.dataframe)


Dados finais após pipeline completo:


,nome_completo,email,telefone,idade,salario,departamento,data_admissao
0,João Silva,JOÃO@EMAIL.COM,(11) 99999-9999,25,5000.0,TI,2020-01-15
1,MARIA SANTOS,maria@email.com,(21) 88888-8888,30,6000.5,RH,2019-03-20
3,Ana Costa,ana@email.com,(31) 77777-7777,35,7500.0,Vendas,2018-06-10
6,Pedro Alves,pedro@email.com,(41) 66666-6666,150,10000.0,TI,2017-12-05


In [13]:
# Célula 10: Análise comparativa
print("Resumo da transformação:")
resumo = pd.DataFrame({
    'Estágio': ['Original', 'Após Cleaning', 'Após Filtering', 'Após Deduplication', 'Final'],
    'Registros': [
        len(df_original),
        len(df_cleaned),
        len(df_filtered), 
        len(df_dedup),
        len(resultado.dataframe)
    ]
})
display(resumo)

Resumo da transformação:


,Estágio,Registros
0,Original,8
1,Após Cleaning,8
2,Após Filtering,6
3,Após Deduplication,4
4,Final,4


In [14]:
# Célula 11: Exportar resultados (opcional)
# resultado.dataframe.to_csv('dados_processados.csv', index=False)
print("Dados exportados para 'dados_processados.csv'")

Dados exportados para 'dados_processados.csv'
